In [ ]:
# User interface components for DSM2 related information
import os
from functools import lru_cache
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
# our imports
import pyhecdss
import pydsm
from pydsm.input import parser
from pydsm import hydroh5
from pydsm.input import parser, network
import networkx as nx
from pydelmod import dsm2ui
from vtools.functions.filter import godin
# viz imports
import geoviews as gv
import hvplot.pandas
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import colorcet as cc
#
import param
import panel as pn
pn.extension()

In [ ]:
node_shapefile = 'd:/delta/maps/v8.2-opendata/gisgridmapv8.2nodes/dsm2_nodes_8_2.shp'
hydro_echo_file = 'd:/delta/DSM2v821/study_templates/historical/output/hydro_echo_hist_v821.inp'

In [ ]:
tables = dsm2ui.load_echo_file(hydro_echo_file)
gc = network.build_network_channels(tables)

In [ ]:
nodes = dsm2ui.load_dsm2_node_shapefile(node_shapefile)

nodes['x']=nodes.geometry.x
nodes['y']=nodes.geometry.y
hv.element.tiles.CartoLight()*nodes.hvplot()

In [ ]:
node_map = dsm2ui.to_node_tuple_map(nodes)

In [ ]:
opts.defaults(opts.Graph(height=800, width=600))

In [ ]:
# should work but doesn't yet
tiled_network = hv.element.tiles.CartoLight()* hv.Graph.from_networkx(gc, node_map).opts(
        opts.Graph(directed=True, 
                   arrowhead_length=0.001, 
                   labelled=['index'],
                   node_alpha=0.5, node_size=10
                  )
    )


In [ ]:
selector = hv.streams.Selection1D(source = tiled_network.Graph.I.nodes)
def print_selected_index(index):
    print(f'Selected {index} on graph')

selector.add_subscriber(print_selected_index)

In [ ]:
# this second part of overlay needed only because of issue. 
# see https://discourse.holoviz.org/t/selection-on-graph-nodes-doesnt-work/3437
tiled_network*(tiled_network.Graph.I.nodes.opts(alpha=0)) 

In [ ]:
node_index = 322

In [ ]:
node_id = int(tiled_network.Graph.I.nodes.data.iloc[node_index].values[2])
node_id

In [ ]:
in_channels, out_channels = dsm2ui.get_in_out_channel_numbers(gc, node_id)

In [ ]:
import pydsm

In [ ]:
io_file_table = tables['IO_FILE']
io_file_table

In [ ]:
ot = tables['OUTPUT_CHANNEL'].copy()

In [ ]:
ot

In [ ]:
from pydelmod.dsm2study import abs_path

In [ ]:
ot['FILE']=ot.apply(lambda r: abs_path(r['FILE'], hydro_echo_file), axis=1)

In [ ]:
hdf5_file_io_row = io_file_table[io_file_table.TYPE=='hdf5']
hdf5_file_io_row.iloc[0]['FILE']

In [ ]:
from pydelmod.dsm2ui import get_hydro_tidefile, abs_path

In [ ]:
hydro_tidefile = abs_path(get_hydro_tidefile(tables), hydro_echo_file)
hydro_tidefile

In [ ]:
hydro = pydsm.hydroh5.HydroH5(hydro_tidefile)

In [ ]:
in_flows = [hydro.get_channel_flow(cid,'upstream',timewindow='01JUL2008 0000 - 01AUG2008 0000') for cid in in_channels]

In [ ]:
out_flows = [hydro.get_channel_flow(cid,'downstream',timewindow='01JUL2008 0000 - 01AUG2008 0000') for cid in out_channels]

In [ ]:
total_inflows = sum([f.iloc[:,0] for f in in_flows])
total_outflows = sum([f.iloc[:,0] for f in out_flows])

In [ ]:
mean_inflows = [df.mean() for df in in_flows]

mean_inflows

In [ ]:
mean_outflows = [df.mean() for df in out_flows]
mean_outflows

In [ ]:
sum([f.values[0] for f in mean_outflows])

In [ ]:
su

In [ ]:
mean_outflows[0].values[0]+mean_outflows[1].values[0]

In [ ]:
in_flows[0].columns[0]

In [ ]:
tsin=[df.hvplot(label=df.columns[0]) for df in in_flows]
tsout=[df.hvplot(label=df.columns[0]) for df in out_flows]

In [ ]:
(hv.Overlay(tsin).opts(title='Inflows')+hv.Overlay(tsout).opts(title='Outflows')).cols(1)

In [ ]:
(total_inflows.hvplot(label='total_inflows')*total_outflows.hvplot(label='total_outflows')).opts(title='Total Inflows & Outflows')

In [ ]:
(total_inflows-total_outflows).hvplot(label='Total Inflows - Outflows')

In [ ]:
from vtools.functions.filter import godin

In [ ]:
godin((total_inflows-total_outflows).to_frame()).hvplot(label='Total Inflows - Outflows (Tidally Filtered)')

In [ ]:
node_id

In [ ]:
type(node_id)

In [ ]:
sflow = hydro.get_input_table('/hydro/input/source_flow')
sflow

In [ ]:
sflow_node = sflow[sflow['node']==node_id]
sflow_node

In [ ]:
def abs_path(dssfile, h5file):
    return os.path.normpath(os.path.join(os.path.dirname(h5file),'../',dssfile))

In [ ]:
def get_data_for_source(sn, hydro_tide_filename):
    datasign = sn.sign
    dssfile = sn.file
    dsspath = sn.path
    dssfile = abs_path(dssfile, hydro_tide_filename)
    return datasign*next(pyhecdss.get_ts(dssfile, dsspath))[0]

In [ ]:
sn0=sflow_node.iloc[0]

In [ ]:
sdata = [get_data_for_source(sn, hydro.h5.filename) for _, sn in sflow_node.iterrows()]

In [ ]:
netsources = sum([df.iloc[:,0] for df in sdata])

In [ ]:
netsources.index = netsources.index.to_timestamp()

In [ ]:
net_filtered_flow=godin((total_inflows-total_outflows).to_frame()).iloc[:,0]
net_filtered_flow

In [ ]:
netsources=netsources[net_filtered_flow.index[0]:net_filtered_flow.index[-1]]
netsources

In [ ]:
netmb=(net_filtered_flow.resample('D').mean()+netsources)

In [ ]:
netmb.hvplot(label='Net Mass Balance')

In [ ]:
print('Mass Balance over the period: ', netmb.sum())

In [ ]:
total_inflows
outflow_pcts = [df.iloc[:,0]/total_inflows for df in out_flows]
outflow_pcts

In [ ]:
out_flows[0].sum()/total_inflows.sum()

In [ ]:
def flow_pct(out_flows, total_inflows):
    t=total_inflows.sum()
    return [df.sum()/t for df in out_flows]

In [ ]:
flow_pct(out_flows, total_inflows)

In [ ]:
def build_sankey(node_id, in_flows, out_flows):
    mean_inflows = [df.mean() for df in in_flows]
    mean_outflows = [df.mean() for df in out_flows]
    inlist = [[x.index[0],str(node_id),x[0]] for x in mean_inflows]
    outlist = [[str(node_id),x.index[0],x[0]] for x in mean_outflows]
    edges = pd.DataFrame(inlist+outlist, columns=['from','to','value'])
    sankey = hv.Sankey(edges, label=f'Flows in/out of {node_id}')
    sankey = sankey.opts(label_position='left', edge_fill_alpha=0.75, edge_fill_color='value', node_alpha=0.5, node_color='index', cmap='blues', colorbar=True)
    return sankey#.opts(frame_width=300, frame_height=300)

In [ ]:
build_sankey(node_id, in_flows, out_flows)

# Observation stations
Lets look at observation stations...

In [ ]:
flow_locations = pd.read_csv('d:/dev/DSM2_Calibration_Notebooks/location_info/calibration_flow_stations.csv')
flow_locations

In [ ]:
# keep only those that have location info
flow_locations = flow_locations[~(flow_locations.Latitude.isna() | flow_locations.Longitude.isna())]
flow_locations = flow_locations.astype({'Latitude':'float','Longitude': 'float'})

In [ ]:
tiled_network*flow_locations.hvplot.points(x='Longitude',y='Latitude',geo=True,color='red')